# Baseline Model
- We will be using the processed CIFAR-10 dataset to train a baseline CNN model.
- In the baseline model, we will be using the labelled dataset only, and will be testing the model on the test dataset

## Importing the Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import accuracy_score, log_loss

import tensorflow as tf
import tensorflow.keras.layers as tfl

## Data Exploration

In [2]:
# Importing the Labelled Dataset
df_train = pd.read_csv("../input/cifar10/train_lab_x.csv")
y_train = pd.read_csv("../input/cifar10/train_lab_y.csv")
df_train = np.array(df_train)
y_train = np.array(y_train)
print(df_train.shape, y_train.shape)

# Reshaping the dataset
df_train = np.reshape(df_train, (-1, 3, 32, 32))
print(df_train.shape)

# Visualizing a single image
ind = 11
example = df_train[ind, : , : , : ]
example = example.transpose((1, 2, 0))
plt.figure(figsize=(1.5, 1.5))
plt.imshow(example)
print(y_train[ind])

## Basic Pre-processing

In [3]:
# Creating a random permutation
perm = np.random.permutation(df_train.shape[0])

# Shuffling the training dataset
df_train = df_train[perm, : , : , : ]
y_train = y_train[perm]

# Reshaping, rescaling and one-hot encoding
df_train = np.transpose(np.array(df_train), (0, 2, 3, 1))
df_train = df_train / 255
y_train_oh = tf.one_hot(np.ravel(y_train), depth = 10)

print(df_train.shape, y_train_oh.shape)

## Creating the Model

In [4]:
def cnn_model(input_shape):
    input_img = tf.keras.Input(shape = input_shape)
    
    Z1 = tfl.Conv2D(8, kernel_size=3, strides=1, padding='valid')(input_img)
    A1 = tfl.ReLU()(Z1)
    P1 = tfl.MaxPool2D(pool_size=2, strides=2, padding='valid')(A1)
    
    Z2 = tfl.Conv2D(16, kernel_size=2, strides=1, padding='valid')(P1)
    A2 = tfl.ReLU()(Z2)
    P2 = tfl.MaxPool2D(pool_size=2, strides=2, padding='valid')(A2)
    
    F1 = tfl.Flatten()(P2)
    Den1 = tfl.Dense(256, activation='relu')(F1)
    Drop1 = tfl.Dropout(0.2)(Den1)
    Den2 = tfl.Dense(64, activation='relu')(Drop1)
    Drop2 = tfl.Dropout(0.2)(Den2)
    outputs = tfl.Dense(10, activation='softmax')(Drop2)
    
    model = tf.keras.Model(inputs = input_img, outputs = outputs)
    return model

## Compiling and Training the Model

In [5]:
conv_model = cnn_model((32, 32, 3))
conv_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
conv_model.summary()

In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices((df_train, y_train_oh)).batch(32)
history = conv_model.fit(train_dataset, epochs = 10)

## Inferencing on the Train/Test Sets

In [7]:
# Importing the Test Dataset
df_test = pd.read_csv("../input/cifar10/test_x.csv")
y_test = pd.read_csv("../input/cifar10/test_y.csv")
df_test = np.array(df_test)
y_test = np.array(y_test)
print(df_test.shape, y_test.shape)

# Reshaping the dataset
df_test = np.reshape(df_test, (-1, 3, 32, 32))
print(df_test.shape)

# Reshaping, rescaling and one-hot encoding
df_test = np.transpose(np.array(df_test), (0, 2, 3, 1))
df_test = df_test / 255
y_test_oh = tf.one_hot(np.ravel(y_test), depth = 10)
print(df_test.shape, y_test_oh.shape)

In [8]:
# Predicting on the Train/Test Datasets
preds_train = conv_model.predict(df_train)
preds_test = conv_model.predict(df_test)

# Finding the Predicted Classes
cls_train = np.argmax(preds_train, axis = 1)
cls_test = np.argmax(preds_test, axis = 1)

In [9]:
# Finding the Train/Test set Loss
print("Log-loss for Train Dataset = ", log_loss(y_train_oh, preds_train))
print("Log-loss for Test Dataset = ", log_loss(y_test_oh, preds_test))
print("Accuracy for Train Dataset = ", accuracy_score(y_train, cls_train))
print("Accuracy for Test Dataset = ", accuracy_score(y_test, cls_test))